# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [17]:
import numpy as np
import pandas as pd



# Challenge 1 - The `stats` Submodule

This submodule contains statistical functions for conducting hypothesis tests, producing various distributions and other useful tools. Let's examine this submodule using the KickStarter dataset. Load the data using the `ks-projects-201801.csv` file

In [18]:
ks = pd.read_csv('../ks-projects-201801.csv')

Now print the `head` function to examine the dataset.

In [19]:
ks.head()


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


Import the `mode` function from `scipy.stats` and find the mode of the `country` and `currency` column.

In [20]:
from scipy.stats import mode

print(mode(ks['country']))

print(mode(ks['currency']))



ModeResult(mode=array(['US'], dtype=object), count=array([292627]))
ModeResult(mode=array(['USD'], dtype=object), count=array([295365]))


The trimmed mean is a function that computes the mean of the data with observations removed. The most common way to compute a trimmed mean is by specifying a percentage and then removing elements from both ends. However, we can also specify a threshold on both ends. The goal of this function is to create a more robust method of computing the mean that is less influenced by outliers. SciPy contains a function called `tmean` for computing the trimmed mean. 

In the cell below, import the `tmean` function and then find the 75th percentile of the `goal` column. Compute the trimmed mean between 0 and the 75th percentile of the column. Read more about the `tmean` function [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.tmean.html#scipy.stats.tmean).

In [21]:
from scipy.stats import tmean

np.quantile(ks['goal'],0)

np.quantile(ks['goal'],0.75)


tmean(ks['goal'], (np.quantile(ks['goal'],0),np.quantile(ks['goal'],0.75)))

4874.150287106898

#### SciPy contains various statistical tests. One of the tests is Fisher's exact test. This test is used for contingency tables. 

The test originates from the "Lady Tasting Tea" experiment. In 1935, Fisher published the results of the experiment in his book. The experiment was based on a claim by Muriel Bristol that she can taste whether tea or milk was first poured into the cup. Fisher devised this test to disprove her claim. The null hypothesis is that the treatments do not affect outcomes, while the alternative hypothesis is that the treatment does affect outcome. To read more about Fisher's exact test, see:

* [Wikipedia's explanation](http://b.link/test61)
* [A cool deep explanation](http://b.link/handbook47)
* [An explanation with some important Fisher's considerations](http://b.link/significance76)

Let's perform Fisher's exact test on our KickStarter data. We intend to test the hypothesis that the choice of currency has an impact on meeting the pledge goal. We'll start by creating two derived columns in our dataframe. The first will contain 1 if the amount of money in `usd_pledged_real` is greater than the amount of money in `usd_goal_real`. We can compute this by using the `np.where` function. If the amount in one column is greater than the other, enter a value of 1, otherwise enter a value of zero. Add this column to the dataframe and name it `goal_met`.

In [22]:
ks['goal_met'] = np.where(ks['usd_pledged_real'] > ks['usd_goal_real'],1,0)

ks.sample(5)



,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,goal_met
25259,1128274635,Dungeon Worlds Part 2,Tabletop Games,Games,USD,2016-03-03,4200.0,2016-02-17 13:59:20,8223.11,successful,46,US,8223.11,8223.11,4200.0,1
219473,2117914416,Death Sentence (Part 1),Graphic Novels,Comics,USD,2016-04-24,7500.0,2016-02-24 07:28:58,112.00,failed,6,US,112.00,112.00,7500.0,0
244791,31504002,Record My First Album,Music,Music,USD,2013-02-13,12000.0,2013-01-04 07:14:34,695.00,failed,16,US,695.00,695.00,12000.0,0
85062,1432651973,Machu Picchu Dice | Vol.2 - World's First 4 El...,Product Design,Design,USD,2017-06-01,20000.0,2017-05-02 03:00:39,34721.00,successful,345,US,7473.00,34721.00,20000.0,1
83368,1423915288,The Life: The Experience,Pop,Music,USD,2014-03-30,30000.0,2014-02-06 23:22:30,5715.00,failed,44,US,5715.00,5715.00,30000.0,0


Next, create a column that checks whether the currency of the project is in US Dollars. Create a column called `usd` using the `np.where` function where if the currency is US Dollars, assign a value of 1 to the row and 0 otherwise.

In [23]:
ks['usd'] = np.where(ks['currency'] == 'USD',1,0)

ks.sample(5)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,goal_met,usd
222430,2133157201,John's Sugaring Co. (Canceled),Fashion,Fashion,USD,2015-05-19,2500.0,2015-03-20 03:37:32,0.0,canceled,0,US,0.0,0.00,2500.00,0,1
305900,627718216,Digital Food Magazine,Food,Food,GBP,2015-08-15,5000.0,2015-07-31 11:27:21,0.0,failed,0,GB,0.0,0.00,7836.25,0,0
278286,485822481,Cotton Flowers,Film & Video,Film & Video,EUR,2015-06-28,3100.0,2015-05-29 18:12:31,139.0,undefined,0,"N,0""",NaN,154.23,3439.74,0,0
317872,689077266,Supreme Clientele: A Capsule Collection,Fashion,Fashion,USD,2013-03-23,3500.0,2013-02-21 03:08:05,371.0,failed,8,US,371.0,371.00,3500.00,0,1
106709,154228592,The Breakaway Chronicles,Young Adult,Publishing,USD,2014-09-27,300.0,2014-08-28 20:30:38,85.0,failed,5,US,85.0,85.00,300.00,0,1


Now create a contingency table using the `pd.crosstab` function in the cell below to compare the `goal_met` and `usd` columns.

Import the `fisher_exact` function from `scipy.stats` and conduct the hypothesis test on the contingency table that you have generated above. You can read more about the `fisher_exact` function [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fisher_exact.html#scipy.stats.fisher_exact). The output of the function should be the odds ratio and the p-value. The p-value will provide you with the outcome of the test.

In [25]:
goals = pd.crosstab(ks['goal_met'],ks['usd'])



from scipy.stats import fisher_exact

fisher_exact(goals)



(1.3791838163150314, 2.994e-320)

# Challenge 2 - The `interpolate` submodule

This submodule allows us to interpolate between two points and create a continuous distribution based on the observed data.

In the cell below, import the `interp1d` function from `scipy.interpolate` and first take a sample of 10 rows from `kickstarter`. 

**Make sure there are not duplicated values in `backers`**

In [ ]:
from scipy.interpolate import interp1d


## MISSING

Next, create a linear interpolation of `usd_pledged_real` as a function of the `backers`. Create a function `f` that generates a linear interpolation of `usd_pledged_real` as predicted by the amount of `backers`.

In [ ]:
# Your code here:



Now create a new variable called `x_new`. This variable will contain all integers between the minimum number of backers in our sample and the maximum number of backers. The goal here is to take the dataset that contains few obeservations due to sampling and fill all observations with a value using the interpolation function. 

Hint: one option is the `np.arange` function.

In [ ]:
# Your code here:



Plot function `f` for all values of `x_new`. Run the code below.

In [ ]:
# Run this code:

%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(x_new, f(x_new))

Next create a function that will generate a cubic interpolation function. Name the function `g`.

In [ ]:
# Your code here:



In [ ]:
# Run this code:

plt.plot(x_new, g(x_new))

# Bonus Challenge - The Binomial Distribution

The binomial distribution allows us to calculate the probability of k successes in n trials for a random variable with two possible outcomes (which we typically label success and failure).  

The probability of success is typically denoted by p and the probability of failure is denoted by 1-p.

The `scipy.stats` submodule contains a `binom` function for computing the probabilites of a random variable with the binomial distribution. You may read more about the binomial distribution [here](http://b.link/binomial55)

* In the cell below, compute the probability that a dice lands on 5 exactly 3 times in 8 tries.


In [ ]:
# Your code here:



* Do a simulation for the last event: do a function that simulate 8 tries and return a 1 if the result is 5 exactly 3 times and 0 if not. Now launch your simulation.

In [ ]:
# Your code here:


* Launch 10 simulations and represent the result in a bar plot. Now launch 1000 simulations and represent it. What do you see?

In [ ]:
# Your code here:
